In [3]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
from scipy import integrate
from scipy import optimize as optimize
from matplotlib import pyplot as plt
from tabulate import tabulate

In [4]:
#Condenser calcs (Nusselt correlation)
"""
*since distillate nearly only isobutanol, heat capacity and latent heat data estimated with pure isobutanol
*enthalpy taken reference to liquid @ 25C
*assume adiabatic
*for plates with chevron angle = 45 degrees
"""
########################################################
Twater_in=25+273.15
Tdist_in=108+273.15
Tdist_out=100+273.15

R=1.15*5.05
n_feed=500*(1000/3600) #mol/s
z_feed=0.4
purity=0.95

#n_water_mol_multiplier=9

v_dist=5 #m/s
v_water=5 #m/s

plate_thickness=1e-3 #m
plate_spacing=5e-3 #m
plate_width=0.5 #m
plate_length=1.5 #m 
film_thickness_iso=1e-3 #m
########################################################
def Cp_water(T):
    return 92.053-3.9953e-02*T-2.1103e-04*T**2+5.3469e-07*T**3 #J/(mol*K)
def Cp_iso_l(T):
    return -368.2900757+9.044231228*T-0.06030195*T**2+0.000171544*T**3-1.68717e-07*T**4 #J/(mol*K)
def Cp_iso_v(T):
    return 65.75234591-0.163518928*T+0.001889988*T**2-3.65483e-06*T**3+3.3539e-09*T**4-1.51691e-12*T**5+2.7101e-16*T**6 #J/(mol*K)
def latent_iso(T):
    return (76.19*(1-T/547.73)**0.505)*1000 #J/mol
def k_iso_l(T):
    return 0.19636-0.00024463*T+1e-07*T**2 #W/(m*K)
def k_iso_v(T):
    return -0.0105252+3.60047e-05*T+1.21228e-07*T**2-4.42513e-11*T**3 #W/(m*K)
def k_water(T):
    return -0.298749+0.00470537*T-5.6209e-06*T**2 #W/(m*K)
def mu_iso_l(T):
    return (10**(-11.9687+2177/T+0.023767*T-2.1427e-05*T**2))*1e-3 #Pa*s
def mu_iso_v(T):
    return (-8.10599+0.287735*T-5.31599e-05*T**2+3.29314e-09*T**3)*1e-7 #Pa*s
def mu_water(T):
    return 10**(-11.6225+1948.96/T+0.0216414*T-1.59901e-05*T**2)*1e-3 #Pa*s
def rho_iso_l(T):
    return (0.27250*0.27519**(-(1-T/547.73)**0.23435))*1000 #kg/m^3
def rho_iso_v(T):
    if (T==108+273.15):
        return 2.55 #kg/m^3
def rho_water(T):
    return 1000 #kg/m^3
MM_iso=74.122/1000 #kg/mol 
MM_water=18.01528/1000 #kg/mol
g=9.807 #m/s^2
#######################################################
"""condenser inlet flowrate"""
y1=purity
x1=1-purity
#             D B
A=np.matrix([[1,1],
             [y1,x1]])
b=np.matrix([n_feed,z_feed*n_feed]).T
ans=np.linalg.solve(A,b)
D,B=ans[0,0],ans[1,0]
V=(D*R)+D #total condenser inlet from top of column
#######################################################
"""minimun ratio of water to distillate flowrate before water boils at 99.61C at 1bar"""
min_flow_ratio=(integrate.quad(Cp_iso_l,Tdist_out,Tdist_in)[0]+latent_iso(Tdist_in))/integrate.quad(Cp_water,298.15,372.76)[0]
n_water_mol_multiplier=min_flow_ratio*1.1
n_water=V*n_water_mol_multiplier
#######################################################
"""water temp out"""
def fn(Twater_out,Tdist_in,Tdist_out):
    return V*(integrate.quad(Cp_iso_l,Tdist_out,Tdist_in)[0]+latent_iso(Tdist_in))-n_water*integrate.quad(Cp_water,298.15,Twater_out)[0]
Twater_out=fsolve(fn,323.15,args=(Tdist_in,Tdist_out))
#######################################################
Dh=2*plate_width*plate_spacing/(plate_width+plate_spacing) #hydraulic diameter
#######################################################
"""condensing exchanger area"""

#enthalpy balance
def fn1(Twater_out1,Tdist_in):
    return V*(latent_iso(Tdist_in))-n_water*integrate.quad(Cp_water,298.15,Twater_out1)[0]
Twater_out1=fsolve(fn1,323.15,args=(Tdist_in,))

#temperatures
T_sat=Tdist_in
T_surface=(Tdist_in+Twater_in)/2
T_film_hot=(T_surface+Tdist_in)/2
T_film_cold=(T_surface+Twater_in)/2

#hot side - distillate (vapour) heat transfer coefficient
k_hot_l=k_iso_l(T_film_hot) #thermal conductivity
mu_hot_l=mu_iso_l(T_film_hot) #dynamic viscosity
rho_hot_v=rho_iso_v(Tdist_in) #density
rho_hot_l=rho_iso_l(T_film_hot) #density
Cp_hot_l=Cp_iso_l(T_film_hot)/MM_iso #heat capacity [J/(kg*K)]
latent_hot=latent_iso(Tdist_in)
h_hot=0.943*((rho_hot_l*g*k_hot_l**3*(rho_hot_l-rho_hot_v)*(latent_hot/MM_iso+(3/8)*Cp_hot_l*(T_sat-T_surface)))/(film_thickness_iso*mu_hot_l*(T_sat-T_surface)))**0.25

#cold side - water heat transfer coefficient
k_cold=k_water(T_film_cold) #thermal conductivity
mu_cold=mu_water(T_film_cold) #dynamic viscosity
rho_cold=rho_water(T_film_cold) #density
Cp_cold=Cp_water(T_film_cold)/MM_water #heat capacity [J/(kg*K)]
Pr_cold=mu_cold*Cp_cold/k_cold #Prandlts
Re_cold=rho_cold*v_water*Dh/mu_cold #Reynolds
Nu_cold=0.036*Re_cold**(4/5)*Pr_cold**(1/3) 
h_cold=Nu_cold*k_cold/Dh #heat transfer coefficient

#table
table_cond_hot=[["HX inlet (isobutanol vapour)",k_hot_l,mu_hot_l,rho_hot_l,rho_hot_v,Cp_hot_l,latent_hot,h_hot]]
table_cond_cold=[["Water:",k_cold,mu_cold,rho_cold,Cp_cold,Re_cold,Pr_cold,Nu_cold,h_cold]]

#area calc
U_cond=1/((1/h_hot)+(1/h_cold))
delT1=Tdist_in-Twater_out1
delT2=Tdist_in-Twater_in
delTm=(delT1-delT2)/np.log(delT1/delT2)
Q_cond=V*latent_iso(Tdist_in)
A_cond=Q_cond/(delTm*U_cond)
#######################################################
"""subcooling exchanger area"""

#enthalpy balance
def fn2(Twater_out2,Twater_out1,Tdist_in,Tdist_out):
    return V*integrate.quad(Cp_iso_l,Tdist_out,Tdist_in)[0]-n_water*integrate.quad(Cp_water,Twater_out1,Twater_out2)[0]
Twater_out2=fsolve(fn2,323.15,args=(Twater_out1,Tdist_in,Tdist_out))

#temperatures
T_surface=(Tdist_in+Twater_out1)/2
T_film_hot=(T_surface+Tdist_in)/2
T_film_cold=(T_surface+Twater_out1)/2

#hot side - distillate (liquid)
k_hot_l=k_iso_l(T_film_hot) #thermal conductivity
mu_hot_l=mu_iso_l(T_film_hot) #dynamic viscosity
rho_hot_l=rho_iso_l(T_film_hot) #density
Cp_hot_l=Cp_iso_l(T_film_hot)/MM_iso #heat capacity [J/(kg*K)]
Pr_hot=mu_hot_l*Cp_hot_l/k_hot_l #Prandlts
Re_hot=rho_hot_l*v_dist*Dh/mu_hot_l #Reynolds
Nu_hot=0.036*Re_hot**(4/5)*Pr_hot**(1/3)
h_hot=Nu_hot*k_hot_l/Dh #heat transfer coefficient

#cold side - water
k_cold=k_water(T_film_cold) #thermal conductivity
mu_cold=mu_water(T_film_cold) #dynamic viscosity
rho_cold=rho_water(T_film_cold) #density
Cp_cold=Cp_water(T_film_cold)/MM_water #heat capacity [J/(kg*K)]
Pr_cold=mu_cold*Cp_cold/k_cold #Prandlts
Re_cold=rho_cold*v_dist*Dh/mu_cold #Reynolds
Nu_cold=0.036*Re_cold**(4/5)*Pr_cold**(1/3)
h_cold=Nu_cold*k_cold/Dh #heat transfer coefficient

#table
table_subc=[["HX inlet (isobutanol liquid)",k_hot_l,mu_hot_l,rho_hot_l,Cp_hot_l,Re_hot,Pr_hot,Nu_hot,h_hot],
            ["Water:",k_cold,mu_cold,rho_cold,Cp_cold,Re_cold,Pr_cold,Nu_cold,h_cold]]
#area calc
U_subc=1/((1/h_hot)+(1/h_cold))
delT1=Tdist_in-Twater_out2
delT2=Tdist_out-Twater_out1
delTm=(delT1-delT2)/np.log(delT1/delT2)
Q_subc=V*integrate.quad(Cp_iso_l,Tdist_out,Tdist_in)[0]
A_subc=Q_subc/(delTm*U_subc)
#######################################################
A_total=A_cond+A_subc
plates_count=A_total/(plate_length*plate_width)
V_total=plate_width*plate_length*(plates_count*(plate_thickness+plate_spacing))
# #######################################################
"""output"""
print("Vapour flowrate=",V,"mol/s or",V*MM_iso,"kg/s")
print("Water flowrate=",n_water,"mol/s or",n_water*MM_water,"kg/s")
print("Minimum ratio of water to HX inlet molar flowrate=",min_flow_ratio)
print("Hydraulic diameter=",Dh,"m")
print("_______________________________________________________________________________________________________________________________________________________________")
print("CONDENSING:")
print(" ")
print(tabulate(table_cond_hot,headers=["k (l) [W/(m*K)]","μ (l) [Pa*s]","ρ (l) [kg/m^3]","ρ (v) [kg/m^3]","Cp (l) [J/(kg*K)]","latent heat [J/kg]","h [W/(m^2*K)]"]))
print(" ")
print(tabulate(table_cond_cold,headers=["k [W/(m*K)]","μ [Pa*s]","ρ [kg/m^3]","Cp [J/(kg*K)]","Re","Pr","Nu","h [W/(m^2*K)]"]))        
print(" ")
print("Water exit temp=",Twater_out1-273.15,"C")
print("Overall heat transfer coefficient (condensing)=",U_cond,"W/(m^2*K)")
print("Required area (condensing)",A_cond,"m^2")
print("_______________________________________________________________________________________________________________________________________________________________")
print("SUBCOOLING:")
print(" ")
print(tabulate(table_subc,headers=["k [W/(m*K)]","μ [Pa*s]","ρ [kg/m^3]","Cp [J/(kg*K)]","Re","Pr","Nu","h [W/(m^2*K)]"]))
print(" ")
print("Water exit temp=",Twater_out2-273.15,"C")
print("Overall heat transfer coefficient (subcooling)=",U_subc,"W/(m^2*K)")
print("Required area (subcooling)",A_subc,"m^2")
print("_______________________________________________________________________________________________________________________________________________________________")
print("TOTAL AREA=",A_total,"m^2")
print("# PLATES=",plates_count)
print("VOLUME=",V_total,"m^3")

Vapour flowrate= 367.68904320987656 mol/s or 27.253847260802466 kg/s
Water flowrate= 3155.9696168676355 mol/s or 56.855676319363184 kg/s
Minimum ratio of water to HX inlet molar flowrate= 7.802961064640232
Hydraulic diameter= 0.009900990099009901 m
_______________________________________________________________________________________________________________________________________________________________
CONDENSING:
 
                                k (l) [W/(m*K)]    μ (l) [Pa*s]    ρ (l) [kg/m^3]    ρ (v) [kg/m^3]    Cp (l) [J/(kg*K)]    latent heat [J/kg]    h [W/(m^2*K)]
----------------------------  -----------------  --------------  ----------------  ----------------  -------------------  --------------------  ---------------
HX inlet (isobutanol vapour)           0.121184     0.000715029           743.284              2.55              3272.89               41767.8          3541.69
 
          k [W/(m*K)]     μ [Pa*s]    ρ [kg/m^3]    Cp [J/(kg*K)]       Re       Pr       Nu   

In [5]:
# if (Re_cold<2e5): #linear
#     Nu_cold=0.664*Re_cold**(1/2)*Pr_cold**(1/3) #Nusselt
# elif (Re_cold>3e6): #turbulent
#     Nu_cold=0.036*Re_cold**(4/5)*Pr_cold**(1/3) 
# else:
#     print("Fluid not linear but turbulence not fully developed")

In [6]:
#Reboiler (Nusselt correlation)
"""
*since bottoms nearly only 1-butanol, all data for bottoms is estimated with pure 1-butanol
*enthalpy taken reference to liquid @ 25C
*assume adiabatic
*steam superheated at 1bar
"""
########################################################
Tsteam_in=400+273.15
Tbot_in=133+273.15

R=1.15*5.00
q=1.438
n_feed=500*(1000/3600) #mol/s
z_feed=0.4
purity=0.95


v_bot=5 #m/s
v_steam=8 #m/s

tube_outside_diam=19.05*1e-3 #m
tube_thickness=2.2*1e-3 #m
tube_length=10 #m
########################################################
def Cp_steam(T):
    return (1.84146-0.000180199*T+9.67874e-007*T**2-3.31917e-10*T**3)*1e3 #J/(kg*K)
def Cp_1but_l(T):
    return 83.877+5.6628e-01*T-1.7208e-03*T**2+2.2780e-06*T**3 #J/(mol*K)
def latent_1but(T):
    return (63.024*(1-T/562.93)**0.318)*1e3 #J/mol
def k_1but_l(T):
    return 0.1871-1.1797e-05*T-3.1997e-07*T**2 #W/(m*K)
def k_steam(T):
    return 0.00561987+1.56992e-05*T+1.01063e-07*T**2-2.42824e-11*T**3 #W/(m*K)
def mu_1but_l(T):
    return 10**(-5.397+1.3256e+03/T+6.2223e-03*T-5.5062e-06*T**2)*1e-3 #Pa*s
def mu_steam(T):
    return (-36.826+4.2900e-01*T-1.6200e-05*T**2)*1e-7 #Pa*s
def rho_1but_l(T):
    return (0.26891*0.26674**(-(1-T/562.93)**0.24570))*1e3 #kg/m^3
def rho_steam(T):
    if (T==400+273.15):
        return 0.3222 #kg/m^3
MM_1but=74.121/1000 #kg/mol 
MM_water=18.01528/1000 #kg/mol
#######################################################
"""reboiler inlet flowrate"""
y1=purity
x1=1-purity
#             D B
A=np.matrix([[1,1],
             [y1,x1]])
b=np.matrix([n_feed,z_feed*n_feed]).T
ans=np.linalg.solve(A,b)
D,B=ans[0,0],ans[1,0]
L=D*R
L_bar=L+(n_feed*q)
L_reboiler=L_bar-B
#######################################################
"""minimun ratio of water to distillate flowrate before steam at same temp as L_reboiler"""
min_flow_ratio=latent_1but(Tbot_in)/(integrate.quad(Cp_steam,Tbot_in,Tsteam_in)[0]*MM_water)
n_steam=min_flow_ratio*L_reboiler*1.1
#######################################################
"""steam temp out"""
def fn(Tsteam_out,Tbot_in,Tsteam_in):
    return L_reboiler*latent_1but(Tbot_in)-n_steam*integrate.quad(Cp_steam,Tsteam_out,Tsteam_in)[0]*MM_water
Tsteam_out=fsolve(fn,500,args=(Tbot_in,Tsteam_in))
#######################################################
"""boiling exchanger area"""

T_bulk_hot=(Tsteam_in+Tsteam_out)/2
T_surface=(T_bulk_hot+Tbot_in)/2

#tube side (hot side) - steam
k_hot=k_steam(T_bulk_hot) #thermal conductivity
mu_hot_bulk=mu_steam(T_bulk_hot)  #dynamic viscosity at bulk
mu_hot_surface=mu_steam(T_surface) #dynamic viscosity at surface
rho_hot=rho_steam(Tsteam_in) #density
Cp_hot=Cp_steam(T_bulk_hot) #heat capacity [J/(kg*K)]
Dh_hot=tube_outside_diam-(2*tube_thickness) #hydraulic diameter
Pr_hot=mu_hot_bulk*Cp_hot/k_hot #Prandlts
Re_hot=rho_hot*v_steam*Dh_hot/mu_hot_bulk #Reynolds
if (Re_hot<2300): #laminar
    Pe_hot=Re_hot*Pr_hot #peclet number
    Nu_hot=1.86*(Pe_hot*Dh_hot/tube_length)**(1/3)*(mu_hot_bulk/mu_hot_surface)**0.14
elif (Re_hot>10e4): #turbulent
    Nu_hot=0.023*Re_hot**0.8*Pr_hot**0.3
else:
    print("Re not in range.")
h_hot=Nu_hot*k_hot/Dh_hot #heat transfer coefficient

#shell side (cold side) - 1-butanol
k_cold=k_1but_l(Tbot_in) #thermal conductivity
mu_cold=mu_1but_l(Tbot_in) #dynamic viscosity
rho_cold=rho_1but_l(Tbot_in) #density
Cp_cold=Cp_1but_l(Tbot_in)/MM_1but #heat capacity [J/(kg*K)]
Dh_cold=tube_outside_diam #hydraulic diameter
Pr_cold=mu_cold*Cp_cold/k_cold #Prandlts
Re_cold=rho_cold*v_bot*Dh_cold/mu_cold #Reynolds
Nu_cold=0.33*Re_cold**0.6*Pr_cold**(1/3)
h_cold=Nu_cold*k_cold/Dh_cold #heat transfer coefficient

#table
table_boil=[["Steam:",k_hot,mu_hot_bulk,rho_hot,Cp_hot,Dh_hot,Pr_hot,Re_hot,Nu_hot,h_hot],
            ["HX inlet (1-butanol):",k_cold,mu_cold,rho_cold,Cp_cold,Dh_cold,Pr_cold,Re_cold,Nu_cold,h_cold]]

#area calc
U_boil=1/((1/h_hot)+(1/h_cold))
delT1=Tsteam_in-Tbot_in
delT2=Tsteam_out-Tbot_in
delTm=(delT1-delT2)/np.log(delT1/delT2)
Q_boil=L_reboiler*latent_1but(Tbot_in)
A_boil=Q_boil/(delTm*U_boil)
#######################################################
"""number tubes"""
tube_circum=np.pi*tube_outside_diam
tube_areas=tube_circum*tube_length
tube_count=A_boil/tube_areas
#######################################################
"""output"""
print("Liquid flowrate=",L_reboiler,"mol/s or",L_bar*MM_1but,"kg/s")
print("Steam flowrate=",n_steam,"mol/s or",n_steam*MM_water,"kg/s")
print("Minimum ratio of steam to HX inlet molar flowrate=",min_flow_ratio)
print("________________________________________________________________________________________________________________________________________")
print("BOILING:")
print(tabulate(table_boil,headers=["k [W/(m*K)]","μ [Pa*s]","ρ [kg/m^3]","Cp [J/(kg*K)]","Dh [m]","Pr","Re","Nu","h [W/(m^2*K)]"]))
print(" ")
print("Steam exit temp=",Tsteam_out-273.15,"C")
print("Overall heat transfer coefficient (boiling)=",U_boil,"W/(m^2*K)")
print("Required area (boiling)",A_boil,"m^2")
print("________________________________________________________________________________________________________________________________________")
print("Number tubes required=",tube_count)
# print("VOLUME=",V_total,"m^3")

Liquid flowrate= 425.41666666666663 mol/s or 37.82344300925925 kg/s
Steam flowrate= 2065.985339417534 mol/s or 37.21930436550191 kg/s
Minimum ratio of steam to HX inlet molar flowrate= 4.414891652214479
________________________________________________________________________________________________________________________________________
BOILING:
                         k [W/(m*K)]     μ [Pa*s]    ρ [kg/m^3]    Cp [J/(kg*K)]    Dh [m]        Pr         Re         Nu    h [W/(m^2*K)]
---------------------  -------------  -----------  ------------  ---------------  --------  --------  ---------  ---------  ---------------
Steam:                     0.0410169  1.95128e-05        0.3222          1981.19   0.01465  0.942505    1935.23    2.64233          7.39797
HX inlet (1-butanol):      0.129527   0.000305996      706.035           2463.98   0.01905  5.82092   219774     952.132         6473.85
 
Steam exit temp= [158.09973053] C
Overall heat transfer coefficient (boiling)= [7.38952908] 

In [7]:
##############################
"""steam price optimisation"""
##############################
Tbot_in=116+273.15

R=1.25*5.05
q=1.3906847587268232
n_feed=500*(1000/3600) #mol/s
z_feed=0.4
purity=0.95
##############################
coal_lhv=30000*1e3 #J/kg
coal_price=409*1e-3 #R/kg
water_price=44.50*1e-3 #R/kg
def Cp_steam(T):
    return (1.84146-0.000180199*T+9.67874e-007*T**2-3.31917e-10*T**3)*1e3*MM_water #J/(mol*K)
def latent_water(T):
    return 52.053*(1-T/647.13)**0.321*1e3 #J/mol
def latent_1but(T):
    return (63.024*(1-T/562.93)**0.318)*1e3 #J/mol
def Cp_water(T):
    return 92.053-3.9953e-02*T-2.1103e-04*T**2+5.3469e-07*T**3 #J/(mol*K)
MM_1but=74.121/1000 #kg/mol 
MM_water=18.01528/1000 #kg/mol
##############################
#reboiler inlet flowrate
y1=purity
x1=1-purity
#             D B
A=np.matrix([[1,1],
             [y1,x1]])
b=np.matrix([n_feed,z_feed*n_feed]).T
ans=np.linalg.solve(A,b)
D,B=ans[0,0],ans[1,0]
L=D*R
L_bar=L+(n_feed*q)
##############################
n_steam_rates=np.arange(116,500,1)
n_steam_in_temps=np.arange(2000,5000,1)
cost_total_pm=[]
##############################
#cost
water_duty=integrate.quad(Cp_water,298.15,372.76)[0]+latent_water(372.76)
for n_steam in n_steam_rates:
    cost_total_pm_inner=[]
    for Tsteam_in in n_steam_in_temps:    
            heat_duty=n_steam*(water_duty+integrate.quad(Cp_steam,372.76,Tsteam_in)[0])
            m_coal=heat_duty/coal_lhv
            coal_cost=m_coal*coal_price
            water_cost=n_steam*MM_water*water_price
            cost_total_pm_inner.append((coal_cost+water_cost)*3600*24*31)
    cost_total_pm.append(cost_total_pm_inner)
##############################


In [8]:
##############################
"""steam price optimisation"""
##############################
Tbot_in=116+273.15
#n_steam_mol_multiplier=5 #mol/s
Tsteam_in=400+273.15
R=1.05*5.05
q=1.3906847587268232
n_feed=500*(1000/3600) #mol/s
z_feed=0.4
purity=0.95
##############################
coal_lhv=30000*1e3 #J/kg
coal_price=409*1e-3 #R/kg
water_price=44.50*1e-3 #R/kg
def Cp_steam(T):
    return (1.84146-0.000180199*T+9.67874e-007*T**2-3.31917e-10*T**3)*1e3*MM_water #J/(mol*K)
def latent_water(T):
    return (52.053*(1-T/647.13)**0.321)*1e3 #J/mol
def latent_1but(T):
    return (63.024*(1-T/562.93)**0.318)*1e3 #J/mol
def Cp_water(T):
    return 92.053-3.9953e-02*T-2.1103e-04*T**2+5.3469e-07*T**3 #J/(mol*K)
MM_1but=74.121/1000 #kg/mol 
MM_water=18.01528/1000 #kg/mol
##############################
#reboiler inlet flowrate
y1=purity
x1=1-purity
#             D B
A=np.matrix([[1,1],
             [y1,x1]])
b=np.matrix([n_feed,z_feed*n_feed]).T
ans=np.linalg.solve(A,b)
D,B=ans[0,0],ans[1,0]
L=D*R
L_bar=L+(n_feed*q)
##############################
n_steam_rates=np.arange(116,500,1)
n_steam_in_temps=np.arange(2000,5000,1)
cost_total_pm=[]
##############################
"""minimun ratio of water to distillate flowrate before steam at same temp as L_bar"""
min_flow_ratio=latent_1but(Tbot_in)/(integrate.quad(Cp_steam,Tbot_in,Tsteam_in)[0])
n_steam=min_flow_ratio*L_bar*1.1
"""steam temp out"""
def fn(Tsteam_out,Tbot_in,Tsteam_in):
    return L_bar*latent_1but(Tbot_in)-n_steam*integrate.quad(Cp_steam,Tsteam_out,Tsteam_in)[0]
Tsteam_out=fsolve(fn,500,args=(Tbot_in,Tsteam_in))
##############################

#cost

#water_duty=integrate.quad(Cp_water,298.15,372.76)[0]+latent_water(372.76)
#heat_duty=n_steam*(water_duty+integrate.quad(Cp_steam,372.76,Tsteam_in)[0])
heat_duty=n_steam*integrate.quad(Cp_steam,Tsteam_out,Tsteam_in)[0]
m_coal=heat_duty/coal_lhv
coal_cost=m_coal*coal_price
water_cost=n_steam*MM_water*water_price
cost_total_pm=(coal_cost+water_cost)
##############################
"""output"""
print("Liquid flowrate=",L_bar,"mol/s or",L_bar*MM_1but,"kg/s")
print("Steam flowrate=",n_steam,"mol/s or",n_steam*MM_water,"kg/s")
print("Minimum ratio of steam to HX inlet molar flowrate=",min_flow_ratio)

print("Cost coal p/s= R",coal_cost)
print("Cost water p/s= R", water_cost)
print("Cost coal p/m= R",coal_cost*3600*24*31)
print("Cost water p/m= R", water_cost*3600*24*31)
print("Cost p/m= R",cost_total_pm*3600*24*31)
print(Tsteam_out-273.15)
print(min_flow_ratio)

Liquid flowrate= 479.55112389724405 mol/s or 35.544808854387625 kg/s
Steam flowrate= 2267.4559389654632 mol/s or 40.84885362812573 kg/s
Minimum ratio of steam to HX inlet molar flowrate= 4.298443853338532
Cost coal p/s= R 0.2835426322402962
Cost water p/s= R 1.817773986451595
Cost coal p/m= R 759440.5861924094
Cost water p/m= R 4868725.845311951
Cost p/m= R 5628166.431504361
[142.73866842]
4.298443853338532
